# Nordjyske

Didnt make subscription as scraped within free period.

In [ ]:

!apt-get update
!apt-get install -y wget unzip libvulkan1


!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f -y


!apt-get install -y xvfb

!pip install selenium chromedriver-autoinstaller

import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,445 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:14 http://a

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
import time
from bs4 import BeautifulSoup
import chromedriver_autoinstaller

chromedriver_autoinstaller.install()
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# WebDriver
driver = webdriver.Chrome(options=chrome_options)

# dismiss popups
def dismiss_popups():
    try:
        time.sleep(15)
        consent_button = driver.find_element(By.CLASS_NAME, 'qc-cmp2-consent-info')
        if consent_button:
            driver.execute_script("arguments[0].click();", consent_button)
            print("Dismissed consent popup.")
            time.sleep(2)

        accept_cookies_button = driver.find_element(By.XPATH, "//button[span[text()='Acceptér alle']]")
        if accept_cookies_button:
            driver.execute_script("arguments[0].click();", accept_cookies_button)
            print("Accepted cookies.")
            time.sleep(2)

    except Exception as e:
        print("No consent pop-up or error occurred:", e)
        pass

def click_show_more(max_clicks=5000): # had max clicks in the start as didnt want to scrape a lot of they scraped wrongly
    last_article_count = 0
    click_count = 0

    while click_count < max_clicks:
        try:
            show_more_button = driver.find_element(By.ID, 'more-results')
            if show_more_button.is_displayed():  # Check if btn is visible
                driver.execute_script("arguments[0].click();", show_more_button)
                print("Clicked 'Show More Results' button, loading more results...")
                time.sleep(5)

                current_article_count = len(driver.find_elements(By.CSS_SELECTOR, 'div#results article'))
                if current_article_count == last_article_count:
                    print("No more new articles, stopping...")
                    break
                last_article_count = current_article_count
                click_count += 1
            else:
                print("The 'Show More Results' button is not visible.")
                break

        except Exception as e:
            print("No more 'Show More Results' button or error occurred:", e)
            break

# scrape all articles url after all results have been loaded
def scrape_all_articles():
    articles = driver.find_elements(By.CSS_SELECTOR, 'div#results article')
    article_urls = ['https://nordjyske.dk' + article.get_attribute('data-url') for article in articles]

    scraped_articles = []

    for article_url in article_urls:
        print(f"Scraping article URL: {article_url}")
        data = scrape_article(article_url)
        if data:
            scraped_articles.append(data)

    return scraped_articles

# scrape individual articles
def scrape_article(article_url):
    try:
        driver.get(article_url)
        time.sleep(2)

        header = driver.find_element(By.CSS_SELECTOR, 'h1.display-1.mb-5').text if driver.find_elements(By.CSS_SELECTOR, 'h1.display-1.mb-5') else 'No header'

        subheader = driver.find_element(By.CSS_SELECTOR, '.fs-3.mb-6').text if driver.find_elements(By.CSS_SELECTOR, '.fs-3.mb-6') else 'No subheader'

        category = driver.find_element(By.CSS_SELECTOR, 'a.btn.btn-sm.btn-outline-primary.mb-5').text if driver.find_elements(By.CSS_SELECTOR, 'a.btn.btn-sm.btn-outline-primary.mb-5') else 'No category'

        content_divs = driver.find_elements(By.CSS_SELECTOR, 'article p')
        content = ' '.join([div.text for div in content_divs]) if content_divs else 'No content'

        date = driver.find_element(By.CSS_SELECTOR, 'div.fs-5.align-self-center').text if driver.find_elements(By.CSS_SELECTOR, 'div.fs-5.align-self-center') else 'No date'

        return {'URL': article_url, 'Header': header, 'Subheader': subheader, 'Category': category, 'Content': content, 'Date': date, 'Search term': search_term}

    except Exception as e:
        print(f"Error scraping {article_url}: {e}")
        return None

# Set the search term variable
search_term = 'ansigtsgenkendelse' # as nordjyske outputtet any subword, all the other terms resulted in too many results. see section on data cleaning for more on this

all_scraped_articles = []

print(f"\nStarting search for: {search_term}")

base_url = f'https://nordjyske.dk/search?query={search_term}'

driver.get(base_url)

dismiss_popups()

click_show_more()

scraped_articles = scrape_all_articles()

all_scraped_articles.extend(scraped_articles)

df = pd.DataFrame(all_scraped_articles)


driver.quit()



Starting search for: genkendelse
No consent pop-up or error occurred: Message: no such element: Unable to locate element: {"method":"css selector","selector":".qc-cmp2-consent-info"}
  (Session info: chrome=129.0.6668.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5644256dd10a <unknown>
#1 0x5644253c35e0 <unknown>
#2 0x564425412be8 <unknown>
#3 0x564425412e81 <unknown>
#4 0x5644254598c4 <unknown>
#5 0x564425437b4d <unknown>
#6 0x564425456d7d <unknown>
#7 0x5644254378c3 <unknown>
#8 0x5644254056b3 <unknown>
#9 0x56442540668e <unknown>
#10 0x5644256a7b0b <unknown>
#11 0x5644256aba91 <unknown>
#12 0x564425694305 <unknown>
#13 0x5644256ac612 <unknown>
#14 0x56442567946f <unknown>
#15 0x5644256cc008 <unknown>
#16 0x5644256cc1d3 <unknown>
#17 0x5644256dbf5c <unknown>
#18 0x7fa825acdac3 <unknown>

Clicked 'Show More Results' button, loading more results...
Clicked 'Show Mor

FileNotFoundError: Cannot find file: nordjyske_scraped_articles.csv.csv

In [ ]:
df#['Content'][22]

,URL,Header,Subheader,Category,Content,Date,Search term
0,https://nordjyske.dk/nyheder/danmark/datatilsy...,Datatilsyn afkræver svar fra politiet om brug ...,Politiet har for nylig fået tilladelse til at ...,POLITI,Politiet har for nylig fået tilladelse til at ...,09. september 2024 kl. 17:21\nOpdateret kl. 17:47,genkendelse
1,https://nordjyske.dk/nyheder/verden/hollandsk-...,Hollandsk vagthund idømmer ansigtsgenkendelses...,Amerikansk firma får endnu en trecifret millio...,BØDE,Amerikansk firma får endnu en trecifret millio...,03. september 2024 kl. 16:58,genkendelse
2,https://nordjyske.dk/nyheder/politik/folketing...,Folketinget tillader politiet brug af ansigtsg...,"Regeringen og folketingspartier har aftalt, at...",POLITI,"Regeringen og folketingspartier har aftalt, at...",05. september 2024 kl. 13:14\nOpdateret kl. 16:18,genkendelse
3,https://nordjyske.dk/nyheder/krimi/politiet-ta...,Politiet tager hul på første projekt med ansig...,Rigspolitiet har hentet tilbud på software til...,POLITI,Rigspolitiet har hentet tilbud på software til...,31. august 2024 kl. 16:05\nOpdateret kl. 16:37,genkendelse
4,https://nordjyske.dk/nyheder/politik/el-bakker...,EL bakker op om brug af ansigtsgenkendelse i s...,Brug af ansigtsgenkendelse i sager om farlig b...,POLITI,Brug af ansigtsgenkendelse i sager om farlig b...,31. august 2024 kl. 23:37,genkendelse
...,...,...,...,...,...,...,...
284,https://nordjyske.dk/nyheder/microsoft-surface...,Microsoft Surface Pro 4: Endnu en stilfuld hybrid,Fjerde afsnit i Microsofts Surface Pro-saga er...,No category,Fjerde afsnit i Microsofts Surface Pro-saga er...,22. december 2015 kl. 16:58,genkendelse
285,https://nordjyske.dk/nyheder/apple-iphone-8-og...,Apple iPhone 8 og 8 Plus: Det fornuftige valg,Apples nye iPhone 8 kommer til at leve i skygg...,TELEFONER,Apples nye iPhone 8 kommer til at leve i skygg...,30. oktober 2017 kl. 14:29,genkendelse
286,https://nordjyske.dk/nyheder/nordjylland/waow-...,Waow - sikke et dyr: Minken fik englevinger og...,"Den var usynlig for det fleste, så var den ove...",MINKAVL,"Den var usynlig for det fleste, så var den ove...",22. november 2020 kl. 19:44\nOpdateret 07. dec...,genkendelse
287,https://nordjyske.dk/plus/vi-har-brug-for-al-d...,"Vi har brug for al den glæde, vi kan få",Helle Skaarup fra Thyholm har været leder af m...,No category,Helle Skaarup fra Thyholm har været leder af m...,18. november 2017 kl. 18:53,genkendelse


In [ ]:
# Save the DataFrame to a CSV file
from google.colab import files

df.to_csv('nordjyske_scraped_articles.csv', index=False)

files.download('nordjyske_scraped_articles.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/Thesis/Data/nordjyske_scraped_articles.csv')
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,URL,Header,Subheader,Category,Content,Date,Search term
0,https://nordjyske.dk/nyheder/danmark/datatilsy...,Datatilsyn afkræver svar fra politiet om brug ...,Politiet har for nylig fået tilladelse til at ...,POLITI,Politiet har for nylig fået tilladelse til at ...,09. september 2024 kl. 17:21\nOpdateret kl. 17:47,genkendelse
1,https://nordjyske.dk/nyheder/verden/hollandsk-...,Hollandsk vagthund idømmer ansigtsgenkendelses...,Amerikansk firma får endnu en trecifret millio...,BØDE,Amerikansk firma får endnu en trecifret millio...,03. september 2024 kl. 16:58,genkendelse
2,https://nordjyske.dk/nyheder/politik/folketing...,Folketinget tillader politiet brug af ansigtsg...,"Regeringen og folketingspartier har aftalt, at...",POLITI,"Regeringen og folketingspartier har aftalt, at...",05. september 2024 kl. 13:14\nOpdateret kl. 16:18,genkendelse
3,https://nordjyske.dk/nyheder/krimi/politiet-ta...,Politiet tager hul på første projekt med ansig...,Rigspolitiet har hentet tilbud på software til...,POLITI,Rigspolitiet har hentet tilbud på software til...,31. august 2024 kl. 16:05\nOpdateret kl. 16:37,genkendelse
4,https://nordjyske.dk/nyheder/politik/el-bakker...,EL bakker op om brug af ansigtsgenkendelse i s...,Brug af ansigtsgenkendelse i sager om farlig b...,POLITI,Brug af ansigtsgenkendelse i sager om farlig b...,31. august 2024 kl. 23:37,genkendelse
...,...,...,...,...,...,...,...
284,https://nordjyske.dk/nyheder/microsoft-surface...,Microsoft Surface Pro 4: Endnu en stilfuld hybrid,Fjerde afsnit i Microsofts Surface Pro-saga er...,No category,Fjerde afsnit i Microsofts Surface Pro-saga er...,22. december 2015 kl. 16:58,genkendelse
285,https://nordjyske.dk/nyheder/apple-iphone-8-og...,Apple iPhone 8 og 8 Plus: Det fornuftige valg,Apples nye iPhone 8 kommer til at leve i skygg...,TELEFONER,Apples nye iPhone 8 kommer til at leve i skygg...,30. oktober 2017 kl. 14:29,genkendelse
286,https://nordjyske.dk/nyheder/nordjylland/waow-...,Waow - sikke et dyr: Minken fik englevinger og...,"Den var usynlig for det fleste, så var den ove...",MINKAVL,"Den var usynlig for det fleste, så var den ove...",22. november 2020 kl. 19:44\nOpdateret 07. dec...,genkendelse
287,https://nordjyske.dk/plus/vi-har-brug-for-al-d...,"Vi har brug for al den glæde, vi kan få",Helle Skaarup fra Thyholm har været leder af m...,No category,Helle Skaarup fra Thyholm har været leder af m...,18. november 2017 kl. 18:53,genkendelse


In [ ]:
df['merged_text'] = df[['Header', 'Subheader', 'Content']].agg(' '.join, axis=1)


In [ ]:

filtered_df = df[df['merged_text'].str.contains(r'ansigtsgenkend\w*', case=False, na=False, regex=True)]
filtered_df['Search term'] = 'ansigtsgenkend*'
len(filtered_df)

<ipython-input-65-eb7874e204c8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Search term'] = 'ansigtsgenkend*'


53

In [ ]:
# fixing date
import re
import pandas as pd


def clean_date_string(date_str):
    match = re.search(r'.*?\d{4}', date_str)
    if match:
        return match.group(0)
    return date_str

filtered_df['Date_cleaned'] = filtered_df['Date'].apply(clean_date_string)

def extract_and_standardize_date(date_str):
    first_date_match = re.search(r'(\d{1,2}\. \w+ \d{4})', date_str)
    if first_date_match:
        first_date_str = first_date_match.group(1)
        try:
            return pd.to_datetime(first_date_str, format='%d. %B %Y', dayfirst=True)
        except ValueError:
            return pd.NaT
    else:
        return pd.NaT

month_mapping = {
    'januar': 'January', 'februar': 'February', 'marts': 'March',
    'april': 'April', 'maj': 'May', 'juni': 'June',
    'juli': 'July', 'august': 'August', 'september': 'September',
    'oktober': 'October', 'november': 'November', 'december': 'December'
}

for danish_month, english_month in month_mapping.items():
    filtered_df['Date_cleaned'] = filtered_df['Date_cleaned'].str.replace(danish_month, english_month)

filtered_df['standard_date'] = pd.to_datetime(filtered_df['Date_cleaned'], format='%d. %B %Y', dayfirst=True, errors='coerce')
filtered_df



<ipython-input-66-558301c9e346>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Date_cleaned'] = filtered_df['Date'].apply(clean_date_string)
<ipython-input-66-558301c9e346>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Date_cleaned'] = filtered_df['Date_cleaned'].str.replace(danish_month, english_month)
<ipython-input-66-558301c9e346>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

,URL,Header,Subheader,Category,Content,Date,Search term,merged_text,Date_cleaned,standard_date
0,https://nordjyske.dk/nyheder/danmark/datatilsy...,Datatilsyn afkræver svar fra politiet om brug ...,Politiet har for nylig fået tilladelse til at ...,POLITI,Politiet har for nylig fået tilladelse til at ...,09. september 2024 kl. 17:21\nOpdateret kl. 17:47,ansigtsgenkend*,Datatilsyn afkræver svar fra politiet om brug ...,09. September 2024,2024-09-09
1,https://nordjyske.dk/nyheder/verden/hollandsk-...,Hollandsk vagthund idømmer ansigtsgenkendelses...,Amerikansk firma får endnu en trecifret millio...,BØDE,Amerikansk firma får endnu en trecifret millio...,03. september 2024 kl. 16:58,ansigtsgenkend*,Hollandsk vagthund idømmer ansigtsgenkendelses...,03. September 2024,2024-09-03
2,https://nordjyske.dk/nyheder/politik/folketing...,Folketinget tillader politiet brug af ansigtsg...,"Regeringen og folketingspartier har aftalt, at...",POLITI,"Regeringen og folketingspartier har aftalt, at...",05. september 2024 kl. 13:14\nOpdateret kl. 16:18,ansigtsgenkend*,Folketinget tillader politiet brug af ansigtsg...,05. September 2024,2024-09-05
3,https://nordjyske.dk/nyheder/krimi/politiet-ta...,Politiet tager hul på første projekt med ansig...,Rigspolitiet har hentet tilbud på software til...,POLITI,Rigspolitiet har hentet tilbud på software til...,31. august 2024 kl. 16:05\nOpdateret kl. 16:37,ansigtsgenkend*,Politiet tager hul på første projekt med ansig...,31. August 2024,2024-08-31
4,https://nordjyske.dk/nyheder/politik/el-bakker...,EL bakker op om brug af ansigtsgenkendelse i s...,Brug af ansigtsgenkendelse i sager om farlig b...,POLITI,Brug af ansigtsgenkendelse i sager om farlig b...,31. august 2024 kl. 23:37,ansigtsgenkend*,EL bakker op om brug af ansigtsgenkendelse i s...,31. August 2024,2024-08-31
5,https://nordjyske.dk/nyheder/krimi/dommere-vil...,Dommere vil have snor i politiets brug af ansi...,Brug af nyt værktøj er ligeså alvorligt et ind...,DOMSTOL,Brug af nyt værktøj er ligeså alvorligt et ind...,26. august 2024 kl. 08:12\nOpdateret kl. 09:01,ansigtsgenkend*,Dommere vil have snor i politiets brug af ansi...,26. August 2024,2024-08-26
6,https://nordjyske.dk/nyheder/politik/hummelgaa...,Minister vægter opklaring tungere end modstand...,Unge svenskere rejser til Danmark og begår hår...,EFTERFORSKNING (KRIMINEL),Unge svenskere rejser til Danmark og begår hår...,12. august 2024 kl. 15:10\nOpdateret kl. 16:18,ansigtsgenkend*,Minister vægter opklaring tungere end modstand...,12. August 2024,2024-08-12
7,https://nordjyske.dk/nyheder/krimi/ansigtsgenk...,Ansigtsgenkendelse kan blive del af kampen mod...,Justitsminister Peter Hummelgaard vil give pol...,FORBRYDELSE,Justitsminister Peter Hummelgaard vil give pol...,08. august 2024 kl. 14:31,ansigtsgenkend*,Ansigtsgenkendelse kan blive del af kampen mod...,08. August 2024,2024-08-08
13,https://nordjyske.dk/nyheder/politik/danmarksd...,Danmarksdemokraterne vil bevogte grænser med 2...,Grænsekontrol skal ifølge DD styrkes med kontr...,POLITIK,Grænsekontrol skal ifølge DD styrkes med kontr...,17. august 2024 kl. 06:00\nOpdateret kl. 06:38,ansigtsgenkend*,Danmarksdemokraterne vil bevogte grænser med 2...,17. August 2024,2024-08-17
16,https://nordjyske.dk/nyheder/politik/rigspolit...,Rigspolitichef frabeder sig grænsekontrol af l...,Det er ifølge rigspolitichefen uinteressant fo...,EFTERFORSKNING (KRIMINEL),Det er ifølge rigspolitichefen uinteressant fo...,12. august 2024 kl. 16:17\nOpdateret kl. 16:40,ansigtsgenkend*,Rigspolitichef frabeder sig grænsekontrol af l...,12. August 2024,2024-08-12


In [ ]:
df = filtered_df.drop(columns=['Date', 'Date_cleaned'])
df.columns = df.columns.str.lower()

df = df.rename(columns={'standard_date': 'date'})

df

,url,header,subheader,category,content,search term,merged_text,date
0,https://nordjyske.dk/nyheder/danmark/datatilsy...,Datatilsyn afkræver svar fra politiet om brug ...,Politiet har for nylig fået tilladelse til at ...,POLITI,Politiet har for nylig fået tilladelse til at ...,ansigtsgenkend*,Datatilsyn afkræver svar fra politiet om brug ...,2024-09-09
1,https://nordjyske.dk/nyheder/verden/hollandsk-...,Hollandsk vagthund idømmer ansigtsgenkendelses...,Amerikansk firma får endnu en trecifret millio...,BØDE,Amerikansk firma får endnu en trecifret millio...,ansigtsgenkend*,Hollandsk vagthund idømmer ansigtsgenkendelses...,2024-09-03
2,https://nordjyske.dk/nyheder/politik/folketing...,Folketinget tillader politiet brug af ansigtsg...,"Regeringen og folketingspartier har aftalt, at...",POLITI,"Regeringen og folketingspartier har aftalt, at...",ansigtsgenkend*,Folketinget tillader politiet brug af ansigtsg...,2024-09-05
3,https://nordjyske.dk/nyheder/krimi/politiet-ta...,Politiet tager hul på første projekt med ansig...,Rigspolitiet har hentet tilbud på software til...,POLITI,Rigspolitiet har hentet tilbud på software til...,ansigtsgenkend*,Politiet tager hul på første projekt med ansig...,2024-08-31
4,https://nordjyske.dk/nyheder/politik/el-bakker...,EL bakker op om brug af ansigtsgenkendelse i s...,Brug af ansigtsgenkendelse i sager om farlig b...,POLITI,Brug af ansigtsgenkendelse i sager om farlig b...,ansigtsgenkend*,EL bakker op om brug af ansigtsgenkendelse i s...,2024-08-31
5,https://nordjyske.dk/nyheder/krimi/dommere-vil...,Dommere vil have snor i politiets brug af ansi...,Brug af nyt værktøj er ligeså alvorligt et ind...,DOMSTOL,Brug af nyt værktøj er ligeså alvorligt et ind...,ansigtsgenkend*,Dommere vil have snor i politiets brug af ansi...,2024-08-26
6,https://nordjyske.dk/nyheder/politik/hummelgaa...,Minister vægter opklaring tungere end modstand...,Unge svenskere rejser til Danmark og begår hår...,EFTERFORSKNING (KRIMINEL),Unge svenskere rejser til Danmark og begår hår...,ansigtsgenkend*,Minister vægter opklaring tungere end modstand...,2024-08-12
7,https://nordjyske.dk/nyheder/krimi/ansigtsgenk...,Ansigtsgenkendelse kan blive del af kampen mod...,Justitsminister Peter Hummelgaard vil give pol...,FORBRYDELSE,Justitsminister Peter Hummelgaard vil give pol...,ansigtsgenkend*,Ansigtsgenkendelse kan blive del af kampen mod...,2024-08-08
13,https://nordjyske.dk/nyheder/politik/danmarksd...,Danmarksdemokraterne vil bevogte grænser med 2...,Grænsekontrol skal ifølge DD styrkes med kontr...,POLITIK,Grænsekontrol skal ifølge DD styrkes med kontr...,ansigtsgenkend*,Danmarksdemokraterne vil bevogte grænser med 2...,2024-08-17
16,https://nordjyske.dk/nyheder/politik/rigspolit...,Rigspolitichef frabeder sig grænsekontrol af l...,Det er ifølge rigspolitichefen uinteressant fo...,EFTERFORSKNING (KRIMINEL),Det er ifølge rigspolitichefen uinteressant fo...,ansigtsgenkend*,Rigspolitichef frabeder sig grænsekontrol af l...,2024-08-12


In [ ]:

from google.colab import files

df.to_csv('nordjyske_scraped_articles_clean.csv', index=False)

files.download('nordjyske_scraped_articles_clean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd

df = pd.read_csv('/content/gdrive/MyDrive/Thesis/Data/Nordjyske/nordjyske_scraped_articles_clean.csv')


'Fogh: Regulering af AI skal bygge på demokratiske værdier Rasmussen Global og softwarevirksomheden Milestone Systems var værter for paneldebat, hvor Anders Fogh talte. Rasmussen Global og softwarevirksomheden Milestone Systems var værter for paneldebat, hvor Anders Fogh talte. Anders Fogh Rasmussen mener, at regulering af kunstig intelligens skal bygges op omkring fundamentale demokratiske værdier og beskyttelse af det enkelte menneskes privatliv. Det fortæller den tidligere dansker statsminister for Venstre, tidligere Nato-generalsekretær og stifter af konsulentvirksomheden Rasmussen Global til Ritzau efter en paneldebat onsdag. - På den anden side vil jeg også advare mod at overregulere. For hvis man forbyder en ny teknologi eller pålægger for kraftige restriktioner på udviklingen, så halter vi bagefter, siger han. Anders Fogh Rasmussen påpeger, at så kan det ende med, at det bliver Kina eller andre, som ikke har demokratiske værdier, der udvikler den nye teknologi. - Min konklusion

In [ ]:
from google.colab import files

df.to_csv('nordjyske.csv', index=False)

files.download('nordjyske.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>